<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/VLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.3 MB/s eta 0:00:00


In [45]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update('jax_enable_x64',True)

from scipy.optimize import root_scalar,root

In [25]:
p=Props(['Ethanol','Water'])
z = jnp.array([1/2,1/2])

In [4]:
def bubblePy_ideal(x, T):
    Pi = x*p.Pvap(T)
    bubbleP = np.sum(Pi)
    return bubbleP, Pi/bubbleP

In [27]:
@jax.jit
def bubbleTy_ideal_eq(x, T, P):
    return jnp.sum(x*p.Pvap(T)/P) - 1.

bubbleTy_ideal_eq_gradT = jax.jit(jax.grad(bubbleTy_ideal_eq, 1))
bubbleTy_ideal_eq_grad2T = jax.jit(jax.grad(bubbleTy_ideal_eq_gradT, 1))



def bubbleTy_ideal(x, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(x*Tb)

    res = root_scalar(lambda T: bubbleTy_ideal_eq(x,T,P), x0=Tguess,
                fprime=lambda T: bubbleTy_ideal_eq_gradT(x,T,P),
                fprime2=lambda T: bubbleTy_ideal_eq_grad2T(x,T,P),
                      method='halley')
    if not(res.converged):
        print(res)
    bubbleT = res.root

    return bubbleT, x*p.Pvap(bubbleT)/P

def dewPx_ideal(y, T):
    y_div_Psat = y/p.Pvap(T)
    dewP = 1./jnp.sum(y_div_Psat)
    return dewP, y_div_Psat*dewP

@jax.jit
def dewTx_ideal_eq(y, T, P):
    return jnp.sum(y*P/p.Pvap(T)) - 1.

dewTx_ideal_eq_gradT = jax.jit(jax.grad(dewTx_ideal_eq, 1))
dewTx_ideal_eq_grad2T = jax.jit(jax.grad(dewTx_ideal_eq_gradT, 1))

def dewTx_ideal(y, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(y*Tb)
    res = root_scalar(lambda T: dewTx_ideal_eq(y,T,P), x0=Tguess,
                fprime=lambda T: dewTx_ideal_eq_gradT(y,T,P),
                fprime2=lambda T: dewTx_ideal_eq_grad2T(y,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    dewT = res.root

    return dewT, y*P/p.Pvap(dewT)

In [6]:
dewTx_ideal(z, 2e5)

Array(380.78240916, dtype=float64)

In [7]:
#NRTL Parameters: Tij = Aij + Bij/T + Cij * Ln(T) + Dij * T (T Deg K)

In [20]:
def NRTL(x, T):

    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
    G = jnp.exp(- p.NRTL_alpha*tau)
    tauG = tau*G

    # xG_einsum = np.einsum('k, ki -> i', x, G)
    # xtauG_einsum = np.einsum('k, ki -> i', x, tauG)
    xG = x @ G
    xtauG = x @ tauG
    xtauG_xG = xtauG / xG
    return jnp.exp(xtauG_xG +  x@  ((G * (tau - xtauG_xG))/xG).T)


In [19]:
def nGexRT(n, T):

    ntot = jnp.sum(n)
    x = n/ntot

    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
    G = jnp.exp(- p.NRTL_alpha*tau)
    tauG = tau*G
    xG = x @ G
    xtauG = x @ tauG
    xtauG_xG = xtauG / xG

    return ntot*jnp.sum(x * xtauG_xG)

lngamma=jax.grad(nGexRT, 0)

def NRTL_Gex(z,T):

    return jnp.exp(lngamma(z,373.15))


In [24]:
@jax.jit
def bubbleTy_eq(x, T, P):
    return jnp.sum(x* NRTL(x,T) * p.Pvap(T)/P) - 1.

bubbleTy_eq_gradT = jax.jit(jax.grad(bubbleTy_eq, 1))
bubbleTy_eq_grad2T = jax.jit(jax.grad(bubbleTy_eq_gradT, 1))



def bubbleTy(x, P):
    T_ideal, y_ideal = bubbleTy_ideal(x, P)

    res = root_scalar(lambda T: bubbleTy_eq(x,T,P), x0=T_ideal,
                fprime=lambda T: bubbleTy_eq_gradT(x,T,P),
                fprime2=lambda T: bubbleTy_eq_grad2T(x,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    bubbleT = res.root

    return bubbleT, x*NRTL(x,bubbleT)*p.Pvap(bubbleT)/P



In [39]:

# @jax.jit
def dewTx_eq(y, vec, P):
    T = vec[0]
    x = vec[1:]**2
    return jnp.r_[x*NRTL(x,T)*p.Pvap(T)/P - y, jnp.sum(x)-1]

dewTx_eq_jac = jax.jacobian(dewTx_eq, 1 )

In [40]:
dewTx_eq(z, jnp.array([350, 0.5, 0.5]), 101325.)

Array([-0.20965995, -0.34660991, -0.5       ], dtype=float64)

In [41]:
dewTx_eq_jac(z, jnp.array([350, 0.5, 0.5]), 101325.)

Array([[0.01151947, 0.84086405, 0.32049616],
       [0.00618735, 0.16932192, 0.44423843],
       [0.        , 1.        , 1.        ]], dtype=float64)

In [42]:
dewT_ideal, dewx_ideal = dewTx_ideal(z,101325)

In [47]:
root(lambda vec: dewTx_eq(z, vec, 101325.), jnp.r_[dewT_ideal, dewx_ideal], jac=lambda vec: dewTx_eq_jac(z, vec, 101325.))

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.287e-11 -1.316e-11 -9.056e-12]
       x: [ 3.575e+02  3.860e-01  9.225e-01]
    nfev: 9
    njev: 1
    fjac: [[-6.688e-01 -7.428e-01 -2.987e-02]
           [ 5.478e-01 -5.196e-01  6.557e-01]
           [ 5.026e-01 -4.222e-01 -7.544e-01]]
       r: [-2.865e-02 -1.084e+00 -9.751e-01  1.125e+00  9.660e-01
           -1.486e+00]
     qtf: [ 6.369e-09 -3.800e-09  1.587e-09]

In [ ]:





dewTx_eq_gradT = jax.jit(jax.grad(dewTx_eq, 1))
dewTx_eq_grad2T = jax.jit(jax.grad(dewTx_eq_gradT, 1))

def dewTx(y, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(y*Tb)
    res = root_scalar(lambda T: dewTx_eq(y,T,P), x0=Tguess,
                fprime=lambda T: dewTx_eq_gradT(y,T,P),
                fprime2=lambda T: dewTx_eq_grad2T(y,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    dewT = res.root

    return dewT, y*P/p.Pvap(dewT)

In [26]:
bubbleTy(z,101325)

(Array(353.02237618, dtype=float64),
 Array([0.65386044, 0.34613956], dtype=float64))

In [ ]:
flash(z, T, P)